# Kalshi KXHIGHLAX Backtesting

This notebook backtests a NO-buying strategy on Kalshi daily temperature contracts (KXHIGHLAX series).

**Pipeline:**
1. Run the XGBoost model on historical features to get adjusted temperature forecasts
2. Fetch historical contract prices from Kalshi
3. Use the forecast + a normal distribution to compute expected values per contract
4. Pick the best NO contract per day and simulate three investment strategies

In [ ]:
import pandas as pd
import numpy as np

DATA_PATH     = "/Users/giulioelmi/Desktop/kelshi_trading/backtesting/backtesting_data.csv"
MODEL_PATH    = "/Users/giulioelmi/Desktop/kelshi_trading/backtesting/best_backtesting.json"
SERIES_TICKER = "KXHIGHLAX"
SIGMA         = 2.5324872296670837  # residual std from model calibration
FEE           = 0.02                # Kalshi trading fee per contract

backtesting_data = pd.read_csv(DATA_PATH, index_col=0)
data_start_date  = backtesting_data["DATE"].min()
data_end_date    = backtesting_data["DATE"].max()
print(f"Data: {data_start_date} -> {data_end_date}  ({len(backtesting_data)} days)")

## 1. Generate Model Predictions

For each historical day, we run the XGBoost model on the pre-engineered features to get an *adjusted forecast* —
the NWS forecast corrected by the model's predicted residual error.

In [ ]:
from model_copy import make_prediction

features = backtesting_data.drop(columns=["DATE", "y", "TMAX_next_d"])

results = [
    {"DATE": backtesting_data["DATE"].iloc[i],
     "adjusted_forecast": make_prediction(features.iloc[i:i+1], MODEL_PATH)}
    for i in range(len(features))
]
results_df = pd.DataFrame(results)
results_df["DATE"] = pd.to_datetime(results_df["DATE"]).dt.date
results_df.head()

## 2. Fetch Historical Market Prices

We pull each event's contracts from the Kalshi API and capture the `yes_ask` price at 12:00 UTC —
the price you would have paid trading at noon on that day.

In [ ]:
from get_market_data import make_daily_request_table, fetch_daily_candles_from_table

req           = make_daily_request_table(SERIES_TICKER, data_start_date, data_end_date, target_hhmm="12:00")
market_prices = fetch_daily_candles_from_table(req)
print(f"Fetched {len(market_prices)} contract-day rows across {market_prices['day'].nunique()} days")
market_prices.head()

## 3. Prepare Market Data

Each contract needs a `floor` and `cap` to define its settlement range:

| Type | Example ticker | Settles YES when | floor | cap |
|------|----------------|------------------|-------|-----|
| B (bucket) | B63.5 | TMAX == 63 | 63.0 | 64.0 |
| T (upper tail) | T73 | TMAX >= 73 | 73.0 | NaN |
| T (lower tail) | T57 | TMAX < 57 | NaN | 57.0 |

For T-type contracts, we determine lower vs upper tail by comparing the threshold against
the bucket range in the same event.

In [ ]:
def add_floor_cap(df):
    out = df.copy()
    out["floor"] = np.nan
    out["cap"]   = np.nan

    # B buckets: settlement range is [threshold-0.5, threshold+0.5)
    b = out["threshold_type"] == "B"
    out.loc[b, "floor"] = out.loc[b, "threshold"] - 0.5
    out.loc[b, "cap"]   = out.loc[b, "threshold"] + 0.5

    # T tails: classify as lower (cap only) or upper (floor only)
    # by comparing the T threshold against the bucket range in its event
    for event, grp in out.groupby("event_ticker"):
        b_thresholds = grp.loc[grp["threshold_type"] == "B", "threshold"]
        t_idx        = grp.index[grp["threshold_type"] == "T"]
        if t_idx.empty or b_thresholds.empty:
            continue
        min_b_floor = b_thresholds.min() - 0.5
        for i in t_idx:
            thr = float(out.loc[i, "threshold"])
            if thr <= min_b_floor + 1e-9:
                out.loc[i, "cap"]   = thr  # lower tail: YES if TMAX < thr
            else:
                out.loc[i, "floor"] = thr  # upper tail: YES if TMAX >= thr

    out["no_ask"] = 1 - out["yes_ask"]
    return out

markets_df = add_floor_cap(market_prices)
markets_df.head()

## 4. Compute Expected Values

For each day, we assume TMAX follows a normal distribution centered on the model's adjusted
forecast with spread `SIGMA`. We apply a **continuity correction** because TMAX is integer-valued:

| Contract type | p_yes formula |
|---|---|
| Bucket `floor=k` | `Phi((k+0.5-mu)/sigma) - Phi((k-0.5-mu)/sigma)` |
| Upper tail `floor=k` | `1 - Phi((k-0.5-mu)/sigma)` |
| Lower tail `cap=k` | `Phi((k-0.5-mu)/sigma)` |

`edge_no_cents = p_no - no_ask` — positive means we have an edge buying NO.

In [ ]:
from model_copy import get_ev

def compute_daily_evs(markets_df, results_df, sigma):
    markets = markets_df.copy()
    markets["day"] = pd.to_datetime(markets["day"]).dt.date
    day_mu = results_df.set_index("DATE")["adjusted_forecast"]

    outputs = []
    for day, group in markets.groupby("day"):
        if day not in day_mu.index:
            continue
        out = get_ev(group, mu=float(day_mu[day]), sigma=sigma)
        out["mu"] = float(day_mu[day])
        outputs.append(out)

    return pd.concat(outputs, ignore_index=True) if outputs else pd.DataFrame()

ev_df = compute_daily_evs(markets_df, results_df, SIGMA)
ev_df[["ticker", "floor", "cap", "yes_ask", "no_ask", "p_yes", "p_no", "edge_no_cents"]].head()

## 5. Select Best Bet and Determine Outcome

Each day we pick the NO contract with the **highest edge** (`edge_no_cents`).

Settlement for NO contracts:
- **Bucket** `floor=k`: NO wins if `TMAX != k` (YES only wins on the exact target temperature)
- **Upper tail** `floor=k`: NO wins if `TMAX < k`
- **Lower tail** `cap=k`: NO wins if `TMAX >= k`

In [ ]:
# Best NO contract per day (one bet per trading day)
best_bets = (
    ev_df
    .loc[ev_df.groupby("day")["edge_no_cents"].idxmax()]
    [["day", "ticker", "floor", "cap", "no_ask", "edge_no_cents"]]
    .rename(columns={"day": "DATE"})
    .assign(DATE=lambda d: pd.to_datetime(d["DATE"]))
    .reset_index(drop=True)
)

# Merge in actual TMAX for settlement
tmax = backtesting_data[["DATE", "TMAX"]].copy()
tmax["DATE"] = pd.to_datetime(tmax["DATE"])

df_pnl = best_bets.merge(tmax, on="DATE")
df_pnl["fee"] = FEE

# Determine whether the NO contract settled as a win
def no_outcome(row):
    tmax, floor, cap = row["TMAX"], row["floor"], row["cap"]
    if pd.notna(floor) and pd.notna(cap):
        return int(tmax != floor)   # bucket: YES wins only if TMAX == floor
    elif pd.notna(floor):
        return int(tmax < floor)    # upper tail: YES wins if TMAX >= floor
    else:
        return int(tmax >= cap)     # lower tail: YES wins if TMAX < cap

df_pnl["outcome"] = df_pnl.apply(no_outcome, axis=1)
df_pnl.head(10)

## 6. Investment Strategies

We simulate three strategies, each investing **$1 per contract**. PnL per bet:

```
pnl = outcome - no_ask - fee
```

| Strategy | Description |
|---|---|
| **1 - Always bet** | Buy the best NO every day, no filter |
| **2 - Price filter** | Only bet when `no_ask < 0.60` (contract costs less than 60 cents) |
| **3 - Edge filter** | Only bet when `edge_no_cents > 0.45` (model shows strong edge) |

In [ ]:
def pnl_summary(df, label):
    if df.empty:
        print(f"--- {label} --- No bets.\n")
        return
    pnl     = df["outcome"] - df["no_ask"] - df["fee"]
    total   = pnl.sum()
    capital = df["no_ask"].sum()
    roc     = total / capital * 100 if capital > 0 else 0
    print(f"--- {label} ({len(df)} bets) ---")
    print(f"  Total PnL:         ${total:.2f}")
    print(f"  Capital deployed:  ${capital:.2f}")
    print(f"  Return on capital: {roc:.1f}%")
    print(f"  Win rate:          {df['outcome'].mean()*100:.0f}%")
    print(f"  Avg edge:          {df['edge_no_cents'].mean():.3f}\n")

# Strategy 1: Always bet on the best NO contract each day
pnl_summary(df_pnl, "Strategy 1: Always bet")

# Strategy 2: Only invest when the contract costs less than 60 cents
pnl_summary(df_pnl[df_pnl["no_ask"] < 0.60], "Strategy 2: Price < 60 cents")

# Strategy 3: Only invest when the model shows a strong edge
pnl_summary(df_pnl[df_pnl["edge_no_cents"] > 0.45], "Strategy 3: Edge > 0.45")